In [1]:
from zipfile import ZipFile
import numpy as np

In [2]:
import pandas as pd
import zipfile
import re

zf = zipfile.ZipFile('Rawdata/gun-violence-data_01-2013_03-2018.csv.zip') 
df = pd.read_csv('Rawdata/gun-violence-data_01-2013_03-2018.csv.zip')

### Drop rows without lat/lon data

In [3]:
df = df.dropna(subset=['latitude'])

### Removing Alaska, Hawaii and District of Columbia

In [4]:
df = df[df['state']!='Alaska']
df = df[df['state']!='Hawaii']
df = df[df['state']!='District of Columbia']

### Format date-time in order to drop year 2013

In [10]:
df['Date'] = pd.to_datetime(df.date.str.strip(), format = '%Y-%m-%d') # make date time
df = df.drop(columns = 'date')

In [11]:
df['Year'] = df.Date.apply(lambda x: x.year)

### Drops irrelevant columns and data from 2013

In [12]:
df = df[df['Year']!= 2013]

df = df.drop(columns = ['address', 'incident_url','source_url','incident_url_fields_missing', 'congressional_district',
                        'gun_stolen','gun_type', 'notes', 'n_guns_involved','sources', 'state_house_district', 'state_senate_district'])


In [13]:
df = df.reset_index()
df = df.drop(columns = 'index')

## Splitting incident characteristics and matching with incident ID 

In [14]:
id_incident = []

for i in range(len(df)):
    id_ = df['incident_id'][i]
    id_2 = np.array(id_)
    incident = df['incident_characteristics'][i]
    if pd.isnull(incident) == True: 
        continue
    else:
        incident2 = np.array([re.split(r'\|\|?',p) for p in re.split(r'\|\|?', incident)]).T
        test = np.c_[id_2, incident2]
        df_test = pd.DataFrame(test)
        id_incident.append(df_test)

               
incidents = pd.concat(id_incident).reset_index().drop('index', 1)

### Count "missing" characteristics to obtain max number of incident characteristics

In [16]:
incidents['MISSING']= incidents.apply(lambda x: x.isnull().sum(), axis='columns')
incidents.loc[incidents['MISSING'] == 0]

,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,MISSING
130518,599153,Shot - Wounded/Injured,"Shot - Dead (murder, accidental, suicide)",Accidental Shooting,Accidental Shooting - Death,Accidental Shooting - Injury,Accidental/Negligent Discharge,Mistaken ID (thought it was an intruder/threat...,Home Invasion,Home Invasion - Resident killed,...,Home Invasion - subject/suspect/perpetrator ki...,Defensive Use,"Defensive Use - Crime occurs, victim shoots su...",Defensive Use - Stand Your Ground/Castle Doctr...,Criminal act with stolen gun,Possession (gun(s) found during commission of ...,Gun(s) stolen from owner,Stolen/Illegally owned gun{s} recovered during...,Concealed Carry License - Victim,0


In [20]:
incidents.columns = ['A','B','C','D','E','F','G','H','I','J','K','L','M','N','O','P','Q','R','S','T','U']

## Identifying the unique values in incident characteristics

In [21]:
stacked = incidents[['B','C','D','E','F','G','H','I','J','K','L','M','N','O','P','Q','R','S','T']].stack()
unikke = stacked.unique()

array(['Shots Fired - No Injuries', 'Officer Involved Incident',
       'Shot - Dead (murder, accidental, suicide)',
       'Shot - Wounded/Injured', 'Drive-by (car to street, car to car)',
       'Armed robbery with injury/death and/or evidence of DGU found',
       'Implied Weapon', 'Home Invasion',
       'Home Invasion - Resident injured', 'Pistol-whipping',
       'Accidental Shooting', 'Accidental Shooting - Injury',
       'Cleaning gun', 'Brandishing/flourishing/open carry/lost/found',
       'Officer Involved Shooting - subject/suspect/perpetrator shot',
       'Possession (gun(s) found during commission of other crimes)',
       'Non-Shooting Incident', 'Drug involvement',
       'ATF/LE Confiscation/Raid/Arrest', 'Unlawful purchase/sale',
       'Car-jacking', 'Accidental Shooting - Death',
       'BB/Pellet/Replica gun', 'Hunting accident', 'Road rage',
       'Suicide^', 'Murder/Suicide', 'Domestic Violence',
       'Home Invasion - No death or injury',
       'Accidental/

### Correct incidents characteristics with problematic formating

In [22]:
unikke2 = ['Shot - Wounded/Injured',
       'Mass Shooting',
       'Possession',
       'Possession of gun by felon or prohibited person',
       'Shot - Dead', 'Gang involvement',
       'Shots Fired - No Injuries',
       'Bar/club incident - in or around establishment',
       'Officer Involved Incident',
       'Officer Involved Shooting - subject/suspect/perpetrator killed',
       'Drug involvement', 'Kidnapping/abductions/hostage',
       'Under the influence of alcohol or drugs',
       'Suicide', 'Murder/Suicide',
       'Attempted Murder/Suicide',
       'Domestic Violence', 'Home Invasion',
       'Home Invasion - Resident killed',
       'Armed robbery with injury/death and/or evidence of DGU found',
       'Drive-by',
       'Officer Involved Shooting - Officer shot',
       'Officer Involved Shooting - Officer killed',
       'Officer Involved Shooting - subject/suspect/perpetrator shot',
       'Officer Involved Shooting - subject/suspect/perpetrator suicide at standoff',
       'Spree Shooting',
       'Institution/Group/Business', 'Animal shot/killed',
       'Home Invasion - Resident injured',
       'Mass Murder',
       'Concealed Carry License - Perpetrator',
       'Stolen/Illegally owned',
       'School Incident', 'School Shooting - university/college',
       'Car-jacking', 'Hate crime', 'House party', 'Defensive Use',
       'Defensive Use - Crime occurs, victim shoots subject/suspect/perpetrator',
       'Defensive Use - Victim stops crime',
       'Workplace shooting',
       'Assault weapon',
       'Pistol-whipping', 'Implied Weapon', 'Accidental Shooting',
       'Accidental Shooting - Injury', 'Cleaning gun',
       'Brandishing/flourishing/open carry/lost/found',
       'Non-Shooting Incident', 'ATF/LE Confiscation/Raid/Arrest',
       'Unlawful purchase/sale', 'Accidental Shooting - Death',
       'BB/Pellet/Replica gun', 'Hunting accident', 'Road rage',
       'Home Invasion - No death or injury',
       'Accidental/Negligent Discharge',
       'Shots fired, no action',
       'Sex crime involving firearm', 'TSA Action',
       'Thought gun was unloaded', 'stolen from owner',
       'Gun range/gun shop/gun show shooting',
       'Home Invasion - subject/suspect/perpetrator killed',
       'Home Invasion - subject/suspect/perpetrator injured',
       'Child Involved Incident', 'Child injured self',
       'Suicide - Attempt', 'Child killed by child',
       'Self-Inflicted',
       'School Shooting - elementary/secondary school',
       'Officer Involved Incident - Weapon involved but no shots fired',
       'Defensive Use - Shots fired, no injury/death',
       'Concealed Carry License - Victim',
       'Officer Involved Shooting - Shots fired, no injury',
       'Accidental Shooting at a Business',
       'Criminal act with stolen gun', 'Child killed self',
       'Child picked up & fired gun',
       'Gun at school, no death/injury - elementary/secondary school',
       'Gun at school, no death/injury - university/college',
       'Non-Aggression Incident', 'Defensive Use - WITHOUT a gun',
       'Police Targeted',
       'Defensive Use - Stand Your Ground/Castle Doctrine established',
       'Playing with gun', 'Child injured by child',
       'Officer Involved Shooting - subject/suspect/perpetrator unarmed',
       'ShotSpotter',
       'Officer Involved Shooting - subject/suspect/perpetrator surrender at standoff',
       'Defensive Use - Good Samaritan/Third Party',
       'LOCKDOWN/ALERT ONLY: No GV Incident Occurred Onsite',
       'Child killed',
       'Child with gun - no shots fired', 'Gun shop robbery or burglary',
       'Shootout',
       'Guns stolen from law enforcement',
       'Defensive use - No shots fired',
       'Officer Involved Shooting - Bystander shot',
       'Officer Involved Shooting - Bystander killed',
       'Child injured', 'Gun buy back action',
       'Terrorism Involvement',
       'Officer Involved Shooting - Accidental discharge - no injury required',
       'Mistaken ID ',
       'Officer Involved Shooting - subject/suspect/perpetrator suicide by cop',
       'Ghost gun', 'Political Violence', 'NAV']

## Creating columns for each incident

In [23]:
for i in unikke2: 
    df[i] = df.incident_characteristics.str.contains(i)

In [38]:
# This dataframe countains nan in incident columns. 
#This is due to the fact that some incidents_characteristics are empty

## Making table with incidents

In [79]:
# Tjekker om vi har identificeret de rigtig variable 
col = [('False',[]),('True',[])]


for i in unikke2:    
    s = df[i].value_counts()
    s1 = list(s)
    for j in range(2): 
        col[j][1].append(s1[j])
    

In [80]:
Dict = {title:column for (title,column) in col}
di = pd.DataFrame(Dict) #Tabel der viser, hvor mange incidents der er 
nv_incident = pd.DataFrame(unikke)
di['Name']=pd.DataFrame(nv_incident, index = di.index)
di = di[['Name','False','True']]


In [81]:
di.sort_values(by=['True'], ascending = False)

,Name,False,True
0,Shots Fired - No Injuries,142045,92206
4,"Drive-by (car to street, car to car)",181716,52535
48,Child injured self,190923,43328
2,"Shot - Dead (murder, accidental, suicide)",195956,38295
6,Implied Weapon,199070,35181
...,...,...,...
104,Mistaken ID (thought it was an intruder/threat...,234171,80
99,Child injured (not child shooter),234220,31
102,Concealed Carry License - Perpetrator,234221,30
107,Political Violence,234238,13


## Saving CSV

In [19]:
df.to_csv('Data/Data_incidents.csv.zip', index = True, sep = ',', compression = 'zip')